In [ ]:
import pytest
pytest.main(['-s'])

In [ ]:
from _pytest.compat import final
help(final)

https://docs.pytest.org/en/reorganize-docs/plugins.html#builtin-plugins

In [ ]:
from _pytest import config
help(config.get_plugin_manager())

In [ ]:
from _pytest import config
config.get_plugin_manager().get_plugins()

In [ ]:
from _pytest import config
dict(config.get_plugin_manager().list_name_plugin())

In [ ]:
from _pytest import config
dir(config.get_plugin_manager().hook)

https://pluggy.readthedocs.io/en/latest/

In [ ]:
import pluggy
pm = pluggy.PluginManager('pytest', implprefix='pytest_')

import _pytest.hookspec
pm.add_hookspecs(_pytest.hookspec)

import _pytest.mark
pm.register(_pytest.mark, 'mark')

import _pytest.python
pm.register(_pytest.python, 'python')

import _pytest.main
pm.register(_pytest.main, 'main')

import _pytest.config
pm.hook.pytest_cmdline_main(config=_pytest.config._prepareconfig(args=[]))

In [ ]:
import pluggy
pm = pluggy.PluginManager('pytest', implprefix='pytest_')

import _pytest.hookspec
pm.add_hookspecs(_pytest.hookspec)

import _pytest.mark
pm.register(_pytest.mark, 'mark')

import _pytest.python
pm.register(_pytest.python, 'python')

import _pytest.main
pm.register(_pytest.main, 'main')

import _pytest.config
cfg = _pytest.config.get_config()
cfg.parse(args=[])
pm.hook.pytest_cmdline_main(config=cfg)

In [ ]:
import pluggy
pm = pluggy.PluginManager('pytest', implprefix='pytest_')

import _pytest.hookspec
pm.add_hookspecs(_pytest.hookspec)

import _pytest.main
pm.register(_pytest.main, 'main')

import _pytest.config
cfg = _pytest.config.get_config()
cfg.parse(args=[])
pm.hook.pytest_cmdline_main(config=cfg)

In [ ]:
import _pytest.config
cfg = _pytest.config.get_config()
cfg.parse(args=[])
cfg.pluginmanager.hook.pytest_cmdline_main(config=cfg)

In [ ]:
cfg.pluginmanager.unregister(name='session')
cfg.pluginmanager.unregister(name='funcmanage')

In [ ]:
import _pytest.config
cfg = _pytest.config.get_config()
cfg.parse(args=[])

import _pytest.main
ss = _pytest.main.Session.from_config(cfg)

import _pytest.python
import py
pc = _pytest.python.PyCollector.from_parent(parent=ss, name='()')

In [ ]:
import _pytest.config
cfg = _pytest.config.get_config()
cfg.parse(args=[])

import _pytest.main
ss = _pytest.main.Session.from_config(cfg)
import _pytest.fixtures
ss._fixturemanager = _pytest.fixtures.FixtureManager(ss)

import _pytest.python
import py
ins = _pytest.python.Instance.from_parent(parent=ss, name='()')

In [ ]:
import _pytest.config
cfg = _pytest.config.get_config()
cfg.parse(args=[])

import _pytest.main
ss = _pytest.main.Session.from_config(cfg)
import _pytest.runner
ss._setupstate = _pytest.runner.SetupState()
import _pytest.fixtures
ss._fixturemanager = _pytest.fixtures.FixtureManager(ss)

import _pytest.python
import py
m = _pytest.python.Module.from_parent(parent=ss, fspath=py.path.local())

In [ ]:
class Object(object):
    def __init__(self, **entries):
        self.__dict__.update(entries)
m.obj = Object(**globals())

In [ ]:
ss.items = m.collect()
_pytest.main.pytest_runtestloop(ss)

In [ ]:
# content of test_time.py

import pytest

from datetime import datetime, timedelta


testdata = [
    (datetime(2001, 12, 12), datetime(2001, 12, 11), timedelta(1)),
    (datetime(2001, 12, 11), datetime(2001, 12, 12), timedelta(-1)),
]


def idfn(val):
    if isinstance(val, (datetime,)):
        # note this wouldn't show any hours/minutes/seconds
        return val.strftime("%Y%m%d")


@pytest.mark.parametrize("a,b,expected", testdata)
def test_timedistance_v0(a, b, expected):
    diff = a - b
    assert diff == expected


@pytest.mark.parametrize("a,b,expected", testdata, ids=["forward", "backward"])
def test_timedistance_v1(a, b, expected):
    diff = a - b
    assert diff == expected


@pytest.mark.parametrize("a,b,expected", testdata, ids=idfn)
def test_timedistance_v2(a, b, expected):
    diff = a - b
    assert diff == expected


@pytest.mark.parametrize(
    "a,b,expected",
    [
        pytest.param(
            datetime(2001, 12, 12), datetime(2001, 12, 11), timedelta(1), id="forward"
        ),
        pytest.param(
            datetime(2001, 12, 11), datetime(2001, 12, 12), timedelta(-1), id="backward"
        ),
    ],
)
def test_timedistance_v3(a, b, expected):
    diff = a - b
    assert diff != expected

In [ ]:
import _pytest.config
cfg = _pytest.config.get_config()
cfg.parse(args=[])

import _pytest.main
ss = _pytest.main.Session.from_config(cfg)
import _pytest.runner
ss._setupstate = _pytest.runner.SetupState()
import _pytest.fixtures
ss._fixturemanager = _pytest.fixtures.FixtureManager(ss)

import _pytest.python
import py
m = _pytest.python.Module.from_parent(parent=ss, fspath=py.path.local())

In [ ]:
class Object(object):
    def __init__(self, **entries):
        self.__dict__.update(entries)
m.obj = Object(**globals())

In [ ]:
import _pytest.runner
c = dict(enumerate(m.collect()))
for i in c:
    print(f'idx = {i}')
    print(_pytest.runner.call_and_report(c[i], 'setup'))
    print(_pytest.runner.call_and_report(c[i], 'call'))
    print(_pytest.runner.call_and_report(c[i], 'teardown', nextitem=c.get(i+1)))

In [ ]:
for i, f in enumerate(m.collect()):
    print(f'idx = {i}')
    f.setup()
    f.runtest()